In [ ]:
import os
import json
import pandas as pd
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

In [ ]:
# Load the answer key to work with
with open(os.path.join(os.getcwd(), "ce311k_answers", "00_variables_raw.json")) as f:
    answer_key = json.load(f)

del answer_key["cells"][0:2]

In [ ]:
# Get the data all cleaned up
questions_ordered = {}
q_num = -1
question, solution, test = False, False, False
for item in answer_key["cells"]:
    # Set the question number
    if item["cell_type"] == "raw":
        if item["source"][0] == "# BEGIN QUESTION\n":
            q_num += 1
    # Ensure naming sequence is created
    if f"q{q_num}" not in questions_ordered.keys():
        questions_ordered[f"q{q_num}"] = {}
        questions_ordered[f"q{q_num}"]["question"] = None
        questions_ordered[f"q{q_num}"]["solution"] = None
        questions_ordered[f"q{q_num}"]["test"] = None
    # Denote what type of info the line is
    if item["cell_type"] == "raw":
        if item["source"][0] == "# BEGIN QUESTION\n":
            question = True
        elif item["source"][0] == "# BEGIN SOLUTION":
            solution = True
        elif item["source"][0] == "# BEGIN TESTS":
            test = True
    # Put the info into the correct place
    elif item["cell_type"] != "raw":
        if question:
            questions_ordered[f"q{q_num}"]["question"] = item["source"]
            question = False
        elif solution:
            questions_ordered[f"q{q_num}"]["solution"] = item["source"][1:-1]
            solution = False
        elif test:
            questions_ordered[f"q{q_num}"]["test"] = item["source"]
            test = False

In [ ]:
# Get data set up into dataframe for vectorization
df = pd.DataFrame(
    {
        "question": [
            questions_ordered[i]["question"] for i in questions_ordered.keys()
        ],
        "solution": [
            questions_ordered[i]["solution"] for i in questions_ordered.keys()
        ],
        "test": [questions_ordered[i]["test"] for i in questions_ordered.keys()],
    }
)

# Define the embedding function
model = SentenceTransformer(
    "all-MiniLM-L6-v2"
)  # This is the default embedding function for chroma

questions = []
solutions = []
tests = []
for i in questions_ordered.keys():
    questions.append(" ".join([str(item) for item in questions_ordered[i]["question"]]))
    solutions.append(" ".join([str(item) for item in questions_ordered[i]["solution"]]))
    tests.append(" ".join([str(item) for item in questions_ordered[i]["test"]]))

questions_concat = []
solutions_concat = []
tests_concat = []

def concatenate(iterable, sep=" "):
    sentence = iterable[0]
    for word in iterable[1:]:
        sentence += (sep + word)
    return sentence

for i in range(len(questions)):
    questions_concat.append(concatenate(questions[i]))
for i in range(len(solutions)):
    solutions_concat.append(concatenate(solutions[i]))
for i in range(len(tests)):
    tests_concat.append(concatenate(tests[i]))

question_embeddings = []
solution_embeddings = []
test_embeddings = []
for i in range(len(questions)):
    question_embeddings.append(model.encode(questions[i]).tolist())
for i in range(len(solutions)):
    solution_embeddings.append(model.encode(solutions[i]).tolist())
for i in range(len(tests)):
    test_embeddings.append(model.encode(tests[i]).tolist())

print(question_embeddings[0])

df["question_embeddings"] = question_embeddings
df["solution_embeddings"] = solution_embeddings
df["test_embeddings"] = test_embeddings

In [ ]:
# Set up the chroma client
chroma_client = chromadb.PersistentClient(path="./chroma_save_states")
collection = chroma_client.get_or_create_collection(name="00_variables")

# Add the data to the chroma client
metas = []
qids = []
sids = []
tids = []
for i in range(len(df.index)):
    metas.append({"00_variables": i})
for i in range(len(df.index)):
    qids.append(f'q{i}')
    sids.append(f's{i}')
    tids.append(f't{i}')

collection.add(
    documents=questions,
    embeddings=question_embeddings,
    ids=qids,
)
collection.add(
    documents=solutions,
    embeddings=df["solution_embeddings"].tolist(),
    metadatas=metas,
    ids=sids,
)
collection.add(
    documents=tests,
    embeddings=df["test_embeddings"].tolist(),
    metadatas=metas,
    ids=tids,
)

In [ ]:
# Query the chromadb and see the context it provides
question = "What variable should I use for question 2?"

results = collection.query(
    [model.encode(question).tolist()],
    n_results=3,
)

# Print the associated answers for the query

# First way
print(collection.query(
    [model.encode(question).tolist()],
    n_results=3,
))

# Second way
collection.get(
    ids=results["ids"][0],
    include=["documents", "embeddings", "metadatas"],
)

In [ ]:
chroma_client.delete_collection("00_variables")